# Topic Model Evaluation

This notebook contains code needed to run the experiments of my Bachelor's thesis on Topic Modeling Algorithms.

For installation please check out `README.md` of this repository if you haven't already.

In [2]:
import sys
sys.path.append('../')

In [3]:
# Import neccessary packages
import numpy as np
from sentence_transformers import SentenceTransformer

from evaluation import DataLoader, Trainer

[nltk_data] Downloading package punkt to /home/budu/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## Trump's tweets

First we will run BERTopic and Top2Vec on Donald Trump's tweets.

His tweets' archive can be found here: https://www.thetrumparchive.com/

For our experiments we will not consider retweets and deleted tweets, those are being filtered out during the operation of `DataLoader` class.

In [3]:
dataset, custom = "trump", True

In [4]:
%%time
dataloader_trump = DataLoader(dataset=dataset).\
                    prepare_docs(save=f"{dataset}.txt").\
                    preprocess_octis(output_folder=f"{dataset}")

100%|█████████████████████████████████████████████████████████████████████████| 46693/46693 [00:00<00:00, 621214.28it/s]


created vocab
53637
words filtering done
CPU times: user 1.29 s, sys: 143 ms, total: 1.44 s
Wall time: 9.33 s


To speed up BERTopic we precalculate embeddings using `SentenceTransformer all-mpnet-base-v2` model. Otherwise, in every different parameter setting would require calculating embeddings, and that would result in a massive runtime.

In [8]:
# Prepare data
dataset, custom = "trump", True
dataloader = DataLoader(dataset)
data_trump = dataloader.load_octis(custom)
data_trump = [" ".join(words) for words in data_trump.get_corpus()]

# Extract embeddings
model = SentenceTransformer("all-mpnet-base-v2")
embeddings_trump = model.encode(data_trump, show_progress_bar=True)

Batches: 100%|██████████████████████████████████████████████████████████████████████| 1383/1383 [13:43<00:00,  1.68it/s]


In [9]:
# Save embeddings
np.savetxt('embeddings_trump.txt', embeddings_trump)

As described in the Thesis and the BERTopic paper [https://arxiv.org/pdf/2203.05794.pdf] the performance of the topic models is reflected by two widely-used metrics, topic coherence and topic diversity.

For each topic model, coherence is evaluated using Normalized Pointwise Mutual Information (NPMI). This measure ranges from $-1$ to $1$, where $1$ indicates a perfect association.

Topic diversity is the percentage of unique words for all topics. This measure ranges from $0$ to $1$ where $0$ indicates redundant topic and $1$ indicates varied topic.

Ranging from $10$ to $50$ topics with steps of $10$, the NPMI score is calculated at each step for each topic model. Results are averaged across $3$ runs for each step.

In [11]:
# Evaluating BERTopic on Trump's tweets

for i in range(3):

    params = {
        "embedding_model": "all-mpnet-base-v2",
        "nr_topics": [(i+1)*10 for i in range(5)],
        "min_topic_size": 15,
        "verbose": True
    }

    trainer = Trainer(dataset=dataset,
                      model_name="BERTopic",
                      params=params,
                      bt_embeddings=embeddings_trump,
                      custom_dataset=custom,
                      verbose=True)
    results = trainer.train(save=f"BERTopic_trump_{i+1}")

2024-03-08 09:04:07,820 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-03-08 09:05:09,432 - BERTopic - Dimensionality - Completed ✓
2024-03-08 09:05:09,435 - BERTopic - Cluster - Start clustering the reduced embeddings
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been us

Results
npmi: 0.04670822821177222
diversity: 0.5888888888888889
 


2024-03-08 09:05:24,489 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-03-08 09:05:53,925 - BERTopic - Dimensionality - Completed ✓
2024-03-08 09:05:53,927 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-03-08 09:05:55,682 - BERTopic - Cluster - Completed ✓
2024-03-08 09:05:55,684 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-03-08 09:05:56,836 - BERTopic - Representation - Completed ✓
2024-03-08 09:05:56,837 - BERTopic - Topic reduction - Reducing number of topics
2024-03-08 09:05:57,718 - BERTopic - Topic reduction - Reduced number of topics from 376 to 20


Results
npmi: -0.030685794454056486
diversity: 0.7263157894736842
 


2024-03-08 09:06:06,855 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-03-08 09:06:33,051 - BERTopic - Dimensionality - Completed ✓
2024-03-08 09:06:33,053 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-03-08 09:06:34,801 - BERTopic - Cluster - Completed ✓
2024-03-08 09:06:34,802 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-03-08 09:06:36,206 - BERTopic - Representation - Completed ✓
2024-03-08 09:06:36,208 - BERTopic - Topic reduction - Reducing number of topics
2024-03-08 09:06:37,237 - BERTopic - Topic reduction - Reduced number of topics from 366 to 30


...

Results
npmi: -0.024701089070270986
diversity: 0.7620689655172413
 


2024-03-08 09:13:22,414 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-03-08 09:13:44,104 - BERTopic - Dimensionality - Completed ✓
2024-03-08 09:13:44,106 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-03-08 09:13:45,692 - BERTopic - Cluster - Completed ✓
2024-03-08 09:13:45,693 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-03-08 09:13:46,690 - BERTopic - Representation - Completed ✓
2024-03-08 09:13:46,692 - BERTopic - Topic reduction - Reducing number of topics
2024-03-08 09:13:47,481 - BERTopic - Topic reduction - Reduced number of topics from 363 to 40


Results
npmi: -0.05100797738638282
diversity: 0.782051282051282
 


2024-03-08 09:13:57,761 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-03-08 09:14:19,133 - BERTopic - Dimensionality - Completed ✓
2024-03-08 09:14:19,136 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-03-08 09:14:20,821 - BERTopic - Cluster - Completed ✓
2024-03-08 09:14:20,823 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-03-08 09:14:21,961 - BERTopic - Representation - Completed ✓
2024-03-08 09:14:21,963 - BERTopic - Topic reduction - Reducing number of topics
2024-03-08 09:14:22,826 - BERTopic - Topic reduction - Reduced number of topics from 379 to 50


Results
npmi: -0.05082242495793191
diversity: 0.7918367346938775
 


In [12]:
# Evaluating Top2Vec on Trump's tweets

for i in range(3):

    params = {"nr_topics": [(i+1)*10 for i in range(5)],
              "hdbscan_args": {'min_cluster_size': 15,
                               'metric': 'euclidean',
                               'cluster_selection_method': 'eom'}}

    trainer = Trainer(dataset=dataset,
                      custom_dataset=custom,
                      custom_model=None,
                      model_name="Top2Vec",
                      params=params,
                      verbose=True)
    results = trainer.train(save=f"Top2Vec_trump_{i+1}")

2024-03-08 09:15:18,223 - top2vec - INFO - Pre-processing documents for training
2024-03-08 09:15:19,369 - top2vec - INFO - Creating joint document/word embedding
2024-03-08 09:20:05,551 - top2vec - INFO - Creating lower dimension embedding of documents
2024-03-08 09:20:19,064 - top2vec - INFO - Finding dense areas of documents
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers

Results
npmi: -0.10390814651919382
diversity: 0.75
 


2024-03-08 09:20:54,672 - top2vec - INFO - Creating joint document/word embedding
2024-03-08 09:25:31,982 - top2vec - INFO - Creating lower dimension embedding of documents
2024-03-08 09:25:48,585 - top2vec - INFO - Finding dense areas of documents
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. D

Results
npmi: -0.16731218688713242
diversity: 0.675
 


2024-03-08 09:26:21,332 - top2vec - INFO - Creating joint document/word embedding
2024-03-08 09:30:49,891 - top2vec - INFO - Creating lower dimension embedding of documents
2024-03-08 09:31:03,919 - top2vec - INFO - Finding dense areas of documents
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. D

...

Results
npmi: -0.13949965709799034
diversity: 0.6766666666666666
 


2024-03-08 10:24:40,914 - top2vec - INFO - Creating joint document/word embedding
2024-03-08 10:29:12,733 - top2vec - INFO - Creating lower dimension embedding of documents
2024-03-08 10:29:26,881 - top2vec - INFO - Finding dense areas of documents
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. D

Results
npmi: -0.20281598009935015
diversity: 0.6575
 


2024-03-08 10:29:59,764 - top2vec - INFO - Creating joint document/word embedding
2024-03-08 10:34:32,379 - top2vec - INFO - Creating lower dimension embedding of documents
2024-03-08 10:34:50,171 - top2vec - INFO - Finding dense areas of documents
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. D

Results
npmi: -0.16828750898234776
diversity: 0.648
 


## 20Newsgroups

The 20Newsgroups dataset comprises 16309 newsgroups posts on 20 topics.

In [13]:
dataset, custom = "20NewsGroup", False
dataloader_20ng = DataLoader(dataset)
data_20ng = dataloader_20ng.load_octis(custom)
data_20ng = [" ".join(words) for words in data_20ng.get_corpus()]

# Extract embeddings
model = SentenceTransformer("all-mpnet-base-v2")
embeddings_20ng = model.encode(data_20ng, show_progress_bar=True)

Batches: 100%|████████████████████████████████████████████████████████████████████████| 510/510 [13:42<00:00,  1.61s/it]


In [14]:
# Save embeddings
np.savetxt('embeddings_20ng.txt', embeddings_20ng)

In [15]:
# Evaluating BERTopic on 20Newsgroups

for i in range(3):

    params = {
        "embedding_model": "all-mpnet-base-v2",
        "nr_topics": [(i+1)*10 for i in range(5)],
        "min_topic_size": 15,
        "verbose": True
    }

    trainer = Trainer(dataset=dataset,
                      model_name="BERTopic",
                      params=params,
                      bt_embeddings=embeddings_20ng,
                      custom_dataset=custom,
                      verbose=True)
    results = trainer.train(save=f"BERTopic_20ng_{i+1}")

2024-03-08 10:49:09,150 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-03-08 10:49:15,984 - BERTopic - Dimensionality - Completed ✓
2024-03-08 10:49:15,986 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-03-08 10:49:16,424 - BERTopic - Cluster - Completed ✓
2024-03-08 10:49:16,425 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-03-08 10:49:17,072 - BERTopic - Representation - Completed ✓
2024-03-08 10:49:17,073 - BERTopic - Topic reduction - Reducing number of topics
2024-03-08 10:49:17,699 - BERTopic - Topic reduction - Reduced number of topics from 83 to 10


Results
npmi: 0.09357326978765458
diversity: 0.8333333333333334
 


2024-03-08 10:49:28,728 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-03-08 10:49:37,055 - BERTopic - Dimensionality - Completed ✓
2024-03-08 10:49:37,057 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-03-08 10:49:37,493 - BERTopic - Cluster - Completed ✓
2024-03-08 10:49:37,495 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-03-08 10:49:38,138 - BERTopic - Representation - Completed ✓
2024-03-08 10:49:38,139 - BERTopic - Topic reduction - Reducing number of topics
2024-03-08 10:49:38,739 - BERTopic - Topic reduction - Reduced number of topics from 85 to 20


Results
npmi: 0.10795244543605588
diversity: 0.7894736842105263
 


2024-03-08 10:49:55,728 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-03-08 10:50:04,130 - BERTopic - Dimensionality - Completed ✓
2024-03-08 10:50:04,132 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-03-08 10:50:04,553 - BERTopic - Cluster - Completed ✓
2024-03-08 10:50:04,554 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-03-08 10:50:05,177 - BERTopic - Representation - Completed ✓
2024-03-08 10:50:05,178 - BERTopic - Topic reduction - Reducing number of topics
2024-03-08 10:50:05,796 - BERTopic - Topic reduction - Reduced number of topics from 82 to 30


...

Results
npmi: 0.12695008613977155
diversity: 0.803448275862069
 


2024-03-08 10:53:11,417 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-03-08 10:53:19,480 - BERTopic - Dimensionality - Completed ✓
2024-03-08 10:53:19,482 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-03-08 10:53:19,928 - BERTopic - Cluster - Completed ✓
2024-03-08 10:53:19,929 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-03-08 10:53:20,498 - BERTopic - Representation - Completed ✓
2024-03-08 10:53:20,499 - BERTopic - Topic reduction - Reducing number of topics
2024-03-08 10:53:21,029 - BERTopic - Topic reduction - Reduced number of topics from 77 to 40


Results
npmi: 0.12256301925854575
diversity: 0.7948717948717948
 


2024-03-08 10:53:30,447 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-03-08 10:53:38,051 - BERTopic - Dimensionality - Completed ✓
2024-03-08 10:53:38,053 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-03-08 10:53:38,477 - BERTopic - Cluster - Completed ✓
2024-03-08 10:53:38,478 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-03-08 10:53:39,096 - BERTopic - Representation - Completed ✓
2024-03-08 10:53:39,098 - BERTopic - Topic reduction - Reducing number of topics
2024-03-08 10:53:39,743 - BERTopic - Topic reduction - Reduced number of topics from 83 to 50


Results
npmi: 0.12048334037241167
diversity: 0.7673469387755102
 


In [16]:
# Evaluating Top2Vec on 20NewsGroups

for i in range(3):

    params = {"nr_topics": [(i+1)*10 for i in range(5)],
              "hdbscan_args": {'min_cluster_size': 15,
                               'metric': 'euclidean',
                               'cluster_selection_method': 'eom'}}

    trainer = Trainer(dataset=dataset,
                      custom_dataset=custom,
                      custom_model=None,
                      model_name="Top2Vec",
                      params=params,
                      verbose=True)
    results = trainer.train(save=f"Top2Vec_20ng_{i+1}")

2024-03-08 10:53:48,212 - top2vec - INFO - Pre-processing documents for training
2024-03-08 10:53:49,389 - top2vec - INFO - Creating joint document/word embedding
2024-03-08 10:55:19,876 - top2vec - INFO - Creating lower dimension embedding of documents
2024-03-08 10:55:25,633 - top2vec - INFO - Finding dense areas of documents
2024-03-08 10:55:25,996 - top2vec - INFO - Finding topics
2024-03-08 10:55:36,993 - top2vec - INFO - Pre-processing documents for training


Results
npmi: 0.19368906248309753
diversity: 0.95
 


2024-03-08 10:55:38,175 - top2vec - INFO - Creating joint document/word embedding
2024-03-08 10:57:10,768 - top2vec - INFO - Creating lower dimension embedding of documents
2024-03-08 10:57:16,744 - top2vec - INFO - Finding dense areas of documents
2024-03-08 10:57:17,100 - top2vec - INFO - Finding topics
2024-03-08 10:57:27,852 - top2vec - INFO - Pre-processing documents for training


Results
npmi: 0.2009235015681318
diversity: 0.905
 


2024-03-08 10:57:29,021 - top2vec - INFO - Creating joint document/word embedding
2024-03-08 10:58:59,640 - top2vec - INFO - Creating lower dimension embedding of documents
2024-03-08 10:59:05,247 - top2vec - INFO - Finding dense areas of documents
2024-03-08 10:59:05,616 - top2vec - INFO - Finding topics
2024-03-08 10:59:17,287 - top2vec - INFO - Pre-processing documents for training


...

Results
npmi: 0.19587179289722034
diversity: 0.8166666666666667
 


2024-03-08 11:17:46,854 - top2vec - INFO - Creating joint document/word embedding
2024-03-08 11:19:25,833 - top2vec - INFO - Creating lower dimension embedding of documents
2024-03-08 11:19:31,655 - top2vec - INFO - Finding dense areas of documents
2024-03-08 11:19:32,011 - top2vec - INFO - Finding topics
2024-03-08 11:19:43,393 - top2vec - INFO - Pre-processing documents for training


Results
npmi: 0.17776161630479406
diversity: 0.735
 


2024-03-08 11:19:44,604 - top2vec - INFO - Creating joint document/word embedding
2024-03-08 11:21:22,284 - top2vec - INFO - Creating lower dimension embedding of documents
2024-03-08 11:21:29,177 - top2vec - INFO - Finding dense areas of documents
2024-03-08 11:21:29,546 - top2vec - INFO - Finding topics


Results
npmi: 0.17568380073170395
diversity: 0.676
 
